In [15]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
     
batch_size = 128
model_name = "Resnet20_quantAware"
model = resnet20_quant() # For quantization aware training


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))     


=> Building model...
Files already downloaded and verified
Files already downloaded and verified


In [16]:
def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [20, 30, 150, 225]
    p_adjust_list = [25, 35]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    if epoch in p_adjust_list:
        for param_group in optimizer.param_groups:
            param_group['momentum'] = param_group['momentum'] / 2

In [ ]:
# This cell won't be given, but students will complete the training

lr = 0.1
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [17]:
model_name = "Resnet20_quantAware"
PATH = "result/" + model_name + "/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8935/10000 (89%)



In [18]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  

In [19]:
######### Save inputs from selected layer ##########
save_output = SaveOutput()
model.layer1[0].conv1.register_forward_pre_hook(save_output) ## Grab input for BasicBlock0 Conv1 layer
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to('cuda')
# Run a forward pass to prehook
out = model(images)

In [20]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
# print(weight_q)
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q / w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 3.0000, -1.0000, -2.0000],
          [-0.0000, -4.0000,  2.0000],
          [ 6.0000, -4.0000,  1.0000]],

         [[ 1.0000,  2.0000, -0.0000],
          [ 0.0000,  1.0000, -2.0000],
          [-2.0000, -1.0000, -4.0000]],

         [[-2.0000, -1.0000,  1.0000],
          [-0.0000, -1.0000,  3.0000],
          [-1.0000,  2.0000,  4.0000]],

         ...,

         [[ 2.0000, -1.0000, -0.0000],
          [-2.0000, -1.0000,  0.0000],
          [ 4.0000, -1.0000, -1.0000]],

         [[ 0.0000,  2.0000,  2.0000],
          [ 0.0000,  1.0000,  1.0000],
          [-1.0000, -0.0000, -0.0000]],

         [[-1.0000,  0.0000,  3.0000],
          [-3.0000,  3.0000,  2.0000],
          [-1.0000,  1.0000,  1.0000]]],


        [[[-1.0000, -0.0000,  3.0000],
          [-1.0000, -1.0000,  2.0000],
          [-1.0000,  1.0000,  0.0000]],

         [[-0.0000,  2.0000, -0.0000],
          [ 0.0000,  1.0000, -2.0000],
          [-3.0000, -2.0000, -2.0000]],

         [[-0.0000,  1.0000, -3

In [21]:
x_bit = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha = model.layer1[0].conv1.act_alpha
x_delta = x_alpha/(2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 6.0000,  5.0000,  5.0000,  ...,  3.0000,  4.0000, 15.0000],
          [ 5.0000,  5.0000,  3.0000,  ...,  3.0000,  3.0000, 15.0000],
          [ 1.0000,  2.0000,  4.0000,  ...,  2.0000,  3.0000, 15.0000],
          ...,
          [ 3.0000,  4.0000,  4.0000,  ...,  3.0000,  3.0000, 15.0000],
          [ 3.0000,  3.0000,  3.0000,  ...,  3.0000,  3.0000, 15.0000],
          [ 1.0000,  3.0000,  3.0000,  ...,  3.0000,  3.0000, 15.0000]],

         [[ 0.0000,  2.0000,  2.0000,  ...,  0.0000,  2.0000,  4.0000],
          [ 0.0000,  3.0000,  3.0000,  ...,  0.0000,  1.0000,  5.0000],
          [ 0.0000,  3.0000,  3.0000,  ...,  0.0000,  1.0000,  5.0000],
          ...,
          [ 0.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  5.0000],
          [ 0.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  5.0000],
          [ 0.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  6.0000]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  3.0000,  4.0000,  3.0000],
          [ 0.0000,  1.0000,  

In [22]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*x_delta*w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[-4.9250e+00, -2.0931e+00,  8.4956e+00,  ...,  3.9400e+01,
           -1.0958e+01, -2.3394e+00],
          [ 3.4228e+01,  2.1300e+01,  2.5364e+01,  ...,  2.8688e+01,
           -5.6637e+00,  9.6037e+00],
          [-7.1412e+00,  1.7237e+01,  1.9577e+01,  ...,  2.7210e+01,
           -1.1451e+01,  9.2343e+00],
          ...,
          [ 6.5256e+00,  1.1697e+01,  6.7718e+00,  ...,  1.7607e+01,
            1.1697e+01,  9.8499e+00],
          [ 2.4748e+01,  1.6006e+01,  1.6991e+01,  ...,  9.3574e+00,
            1.6991e+01,  9.1112e+00],
          [-1.2928e+01, -9.1112e+00, -7.7568e+00,  ..., -5.6637e+00,
           -6.8950e+00, -1.1204e+01]],

         [[-2.2039e+01, -1.8961e+01, -2.1054e+01,  ..., -2.3270e+01,
           -3.5583e+01, -1.2805e+01],
          [-1.1327e+01, -1.2436e+01, -1.5391e+01,  ..., -1.9823e+01,
           -3.0781e+01, -1.3297e+01],
          [-1.6499e+01, -1.1697e+01, -1.0958e+01,  ..., -6.0331e+00,
           -2.1916e+01, -1.4159e+01],
          ...,
     

In [23]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q
print(conv_ref.weight)
output_ref = conv_ref(x)
print(output_ref)

Parameter containing:
tensor([[[[ 1.2169, -0.4056, -0.8113],
          [-0.0000, -1.6225,  0.8113],
          [ 2.4338, -1.6225,  0.4056]],

         [[ 0.4056,  0.8113, -0.0000],
          [ 0.0000,  0.4056, -0.8113],
          [-0.8113, -0.4056, -1.6225]],

         [[-0.8113, -0.4056,  0.4056],
          [-0.0000, -0.4056,  1.2169],
          [-0.4056,  0.8113,  1.6225]],

         ...,

         [[ 0.8113, -0.4056, -0.0000],
          [-0.8113, -0.4056,  0.0000],
          [ 1.6225, -0.4056, -0.4056]],

         [[ 0.0000,  0.8113,  0.8113],
          [ 0.0000,  0.4056,  0.4056],
          [-0.4056, -0.0000, -0.0000]],

         [[-0.4056,  0.0000,  1.2169],
          [-1.2169,  1.2169,  0.8113],
          [-0.4056,  0.4056,  0.4056]]],


        [[[-0.4056, -0.0000,  1.2169],
          [-0.4056, -0.4056,  0.8113],
          [-0.4056,  0.4056,  0.0000]],

         [[-0.0000,  0.8113, -0.0000],
          [ 0.0000,  0.4056, -0.8113],
          [-1.2169, -0.8113, -0.8113]],

         

In [24]:
difference = abs( output_ref - output_recovered )
# diff_percent = 100*abs(output_ref - output_recovered)/output_ref
print(difference.mean().item())  ## It should be small, e.g.,2.3 in my trainned model

1.5333198308944702


In [25]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight_q
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-4.4332e+00, -1.8237e+00,  8.6330e+00,  ...,  6.7205e+01,
           -2.6029e+01, -1.5866e+00],
          [ 3.3416e+01,  2.0932e+01,  2.5067e+01,  ...,  3.5176e+01,
           -1.6901e+01,  9.3242e+00],
          [-5.6278e+00,  1.7097e+01,  1.8710e+01,  ...,  3.4745e+01,
           -2.0214e+01,  1.0302e+01],
          ...,
          [ 7.2409e+00,  1.2737e+01,  6.2227e+00,  ...,  1.7147e+01,
            1.2611e+01,  1.0244e+01],
          [ 2.5424e+01,  1.6970e+01,  1.7890e+01,  ...,  9.9436e+00,
            1.7035e+01,  9.7982e+00],
          [-1.3273e+01, -9.1215e+00, -8.3829e+00,  ..., -6.1064e+00,
           -7.3296e+00, -1.1657e+01]],

         [[-2.1899e+01, -1.9210e+01, -2.1137e+01,  ..., -2.4829e+01,
           -4.4644e+01, -6.4862e+00],
          [-1.1741e+01, -1.3132e+01, -1.4845e+01,  ..., -2.1112e+01,
           -3.6850e+01, -8.3218e+00],
          [-1.6686e+01, -1.1397e+01, -1.0291e+01,  ..., -6.0643e+00,
           -2.7339e+01, -9.0430e+00],
          ...,
     

In [26]:
difference = abs( output_ref - output_recovered )
print(difference.mean().item())  ## It should be small, e.g.,2.3 in my trainned model

1.5167776346206665


In [ ]:
# Start process for 2bit

model_name = "Resnet20_quantAware_2bit"
model = resnet20_quant_2bit() # For quantization aware training

# Train the model
lr = 0.1
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [14]:
model_name = "Resnet20_quantAware_2bit"
model = resnet20_quant_2bit() # For quantization aware training
PATH = "result/" + model_name + "/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7481/10000 (75%)

